In [1]:
import os
import sys
import pandas as pd
import dtale

In [2]:
# pip install dtale

In [3]:
file_path = r'data\EMAILSAMPLEDATA.parquet'
df = pd.read_parquet(file_path)
print(df.shape)

(4901, 9)


In [4]:
# d = dtale.show(df)
# d.open_browser()

In [11]:
# pip install tabula-py

In [5]:
df.head(2)

,id,from_email,to_email,cc_email,subject,date,body,date_ist,time_ist
0,15d7e872-3ed2-4d41-b6a0-515e7b0effdf,EhudR@afcon.co.il,soumendrar@pinnacleinfotech.com,"Joel.adonaBimspecialistphils@outlook.com, Siva...",RE: IFF0203,"Thu, 25 Apr 2024 12:38:36 +0000","1B-1-TB/11-050,051 tack example for wall surfa...",2024-04-25,18:08:46
1,48d1d13f-28f8-4db2-87e4-ac9125ee812c,Dcaf@tatatel.co.in,SUDIPR@PINNACLEINFOTECH.COM,"krishnadas.poddar.kdb@tatatel.co.in, SUDIP.SAH...",Your CAF WBG-8631700017209 and Company Name : ...,"Tue, 23 Apr 2024 14:48:30 +0000 (UTC)",<html>\n<head>\n\t<title>9_SLA mailer-01</titl...,2024-04-23,20:18:36


In [19]:
# pip install beautifulsoup4

In [6]:
import re
from bs4 import BeautifulSoup

In [7]:
def remove_disclaimer_text(body):
    """
    Remove disclaimer and classification notices from the email body.

    Args:
        body (str): The body of the email.
        
    Returns:
        str: The email body without disclaimer and classification notices.
    """
    disclaimer_pattern = [
        r"\*This e-mail has been classified as.*",
        r".*classification notice.*",
        r"From:\s.*Sent:\s.*To:\s.*"
    ]
    
    for pattern in disclaimer_pattern:
        body = re.sub(pattern, '', body, flags=re.DOTALL)
    
    return body

def extract_recent_email(email_body):
    """
    Extract the most recent part of an email thread above the signature or prior email content.

    Args:
        email_body (str): The body of the email.
        
    Returns:
        str: The most recent part text.
    """
    # Use BeautifulSoup to handle HTML email bodies.
    soup = BeautifulSoup(email_body, "html.parser")
    text = soup.get_text()
    lines = text.splitlines()

    # Regex patterns for detecting common signature lines and previous messages
    signature_patterns = [
        re.compile(r'^(-{2,}|_{2,})'),  # Signatures, like "-----Original Message-----" or "__"
        re.compile(r'^\s*(Sent from my [\w\s]+|Sent with [\w\s]+)$', re.IGNORECASE), # Mobile signatures
        re.compile(r'^On \d{4}/[0-1]\d/[0-3]\d,.*'),  # Replies, like "On 2021/12/01, ... wrote:"
        re.compile(r'^\s*From:\s*.*', re.IGNORECASE), # From header
        re.compile(r'^\s*Sent:\s*.*', re.IGNORECASE), # Sent header
        re.compile(r'^To:\s*.*', re.IGNORECASE),      # To header
        re.compile(r'^Subject:\s*.*', re.IGNORECASE), # Subject header
    ]

    # Find the position of signature or previous message markers
    for i, line in enumerate(lines):
        for pattern in signature_patterns:
            if pattern.match(line):
                return "\n".join(lines[:i]).strip()  # Return everything before the signature marker

    return "\n".join(lines).strip()

def extract_text_above_signature(body, from_email):
    """
    Extract text above the signature in the email body.

    Args:
        body (str): The body of the email.
        from_email (str): The sender's email address.
        
    Returns:
        str: The extracted text above signature patterns.
    """
    username = from_email.split('@')[0]  # Extract the username
    signature_patterns = [
        r"\nfrom:\s", r"\nsent:\s", r"\nto:\s", r"\ncc:\s", r"\nsubject:\s",
        r"\nregards,\s", r"\nbest,\s", r"\nthanks,\s", r"\nsincerely,\s", r"\ncheers,\s",
        re.escape(username.lower())
    ]
    
    body_lower = body.lower()
    signature_indices = []
    
    # Find all occurrences of the signature patterns
    for pattern in signature_patterns:
        matches = list(re.finditer(pattern, body_lower, re.MULTILINE))
        signature_indices.extend([match.start() for match in matches])
    
    if signature_indices:
        signature_index = min(signature_indices)
        return body[:signature_index].strip()
    return body

def clean_email_body(body):
    """
    Clean the email body by removing unwanted text based on various regex patterns.

    Args:
        body (str): The body of the email.
    
    Returns:
        str: The cleaned email body.
    """
    patterns = {
        'urls': r'https?://\S+|www\.\S+',  # URLs
        'metadata': r'(?m)^(From|Sent|To|Cc|Bcc|Subject|Date): .*$',  # Email metadata
        'greetings': r'(?i)^(Hi|Hello|Dear|Greetings|Hey)\s+[^\n]+',  # Greetings
        'signatures': r'(?i)(Best regards|Kind regards|Regards|Cheers|Thank you|Thanks|Sincerely|Yours truly|Yours sincerely|Best|Warm regards|With regards)[^\n]+',  # Signatures
        'email_headers': r'---* Forwarded message ---*|---* Original message ---*|---* Reply Above This Line ---*',  # Email forwarding/reply headers
        'email_addresses': r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',  # Email addresses
        'non_ascii': r'[^\x00-\x7F]+',  # Non-ASCII characters
        'extra_lines': r'\n{2,}',  # Excessive newlines
        'html_tags': r'<[^>]*>',  # HTML tags
        'brackets_content': r'\[.*?\]|\{.*?\}|\<.*?\>',  # Content within brackets
        'extra_whitespace': r'\s{2,}',  # Excessive whitespace
        'unsubscribe_links': r'Unsubscribe\s+:.*|Click\s+here.*unsubscribe',  # Unsubscribe links
        'reply_lines': r'On\s+.*wrote:',  # Lines indicating the start of a reply
        'quoted_text': r'(?m)^\>.*$',  # Lines starting with ">"
        'repeated_chars': r'(.)\1{2,}',  # Repeated characters
        'generic_signature_lines': r'(?i)(^Sent from my \w+)|(--\n.*)|(Confidentiality Notice.*)',  # Generic email signatures or legal disclaimers
        'timestamps': r'\d{1,2}:\d{2}\s*(AM|PM|am|pm)?',  # Time stamps
        'dates': r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}',  # Dates
        'signature_blocks': r'--\s*\n[\s\S]*',  # Signature blocks that start with -- followed by any text
        'headers_and_footers': r'(?m)^\s*-\s*$\n[\s\S]*?^\s*-\s*$',  # Headers and footers denoted by lines consisting of dashes
        'mobile_numbers': r'\b(\+?(\d{1,3})?[-. ]?)?((\(\d{1,4}\))|\d{1,4})[-. ]?\d{1,4}[-. ]?\d{1,9}\b'  # Mobile numbers
    }
    
    for key, pattern in patterns.items():
        body = re.sub(pattern, ' ', body)
    
    # Remove excess whitespace
    body = re.sub(r'\s{2,}', ' ', body)
    return body.strip()


def extract_most_recent_email_part(email_body, from_email):
    """
    Combine recent email extraction and signature pattern extraction for robust extraction.

    Args:
        email_body (str): The body of the email.
        from_email (str): The sender's email address.
        
    Returns:
        str: The extracted relevant part of the email above signatures and prior threads.
    """
    # Remove disclaimer and notice sections
    cleaned_body = remove_disclaimer_text(email_body)
    
    # Extract the most recent email section ignoring previous emails
    recent_email_content = extract_recent_email(cleaned_body)
    
    # Extract text above signatures and replies
    filtered_content_above_signature = extract_text_above_signature(recent_email_content, from_email)

    # Clean the email body
    cleaned_body = clean_email_body(filtered_content_above_signature)
    
    return filtered_content_above_signature

In [55]:
# test = df.head(5).copy()

In [8]:
# # Apply the extraction function
# test['clean_email'] = test['body'].apply(extract_recent_email)

# Apply the extraction function
df['clean_email'] = df.apply(lambda row: extract_most_recent_email_part(row['body'], row['from_email']), axis=1)


C:\Users\pis05408.PINNACLE\AppData\Local\Temp\ipykernel_24332\449057951.py:33: MarkupResemblesLocatorWarning:

The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.



In [10]:
# Display the result
print(df['clean_email'][70])

Dear Sir/Madam,

Greetings! I hope this message finds you well. I am


In [50]:
# d = dtale.show(test)
# d.open_browser()

### Sentiment Analysis

In [24]:
from transformers import pipeline

# Define the sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model = "distilbert-base-uncased-finetuned-sst-2-english")

In [104]:
# # Function to split text into chunks
# def split_text_into_chunks(text, max_length=512):
#     return [text[i:i+max_length] for i in range(0, len(text), max_length)]

# # Function to predict sentiment on text chunks
# def predict_sentiment_chunks(text):
#     chunks = split_text_into_chunks(text)
#     sentiments = [sentiment_pipeline(chunk)[0] for chunk in chunks]
#     labels = [sent['label'] for sent in sentiments]
#     scores = [sent['score'] for sent in sentiments]
    
#     # Majority vote for label
#     label = max(set(labels), key=labels.count)
    
#     # Average score for the majority label
#     score = sum([score for i, score in enumerate(scores) if labels[i] == label]) / labels.count(label)
    
#     return {'label': label, 'score': score}

In [25]:

# Function to predict sentiment with text truncation
def predict_sentiment_truncated(text, max_length=512):
    truncated_text = text[:max_length]
    result = sentiment_pipeline(truncated_text)
    return result[0] if result else {'label': 'UNKNOWN', 'score': 0.0}

In [26]:
# Apply the sentiment prediction function
df['sentiment_distilbert'] = df['clean_email'].apply(predict_sentiment_truncated)

In [27]:
# Extract the sentiment label and score into separate columns
df['sentiment_distilbert_label'] = df['sentiment_distilbert'].apply(lambda x: x['label'])
df['sentiment_distilbert_score'] = df['sentiment_distilbert'].apply(lambda x: x['score'])

In [28]:
df.columns

Index(['id', 'from_email', 'to_email', 'cc_email', 'subject', 'date', 'body',
       'date_ist', 'time_ist', 'clean_email', 'sentiment_1',
       'sentiment_transformer_label', 'sentiment_transformer_score',
       'sentiment', 'sentiment_blob_level', 'sentiment_blob_score',
       'sentiment_distilbert', 'sentiment_distilbert_label',
       'sentiment_distilbert_score'],
      dtype='object')

In [31]:
df['sentiment_distilbert_label'].value_counts()

sentiment_distilbert_label
NEGATIVE    3156
POSITIVE    1745
Name: count, dtype: int64

In [117]:
d = dtale.show(df)
d.open_browser()

In [138]:
# pip install textblob

### TextBlob

In [18]:
from textblob import TextBlob

In [19]:
# Function to predict sentiment using TextBlob
def predict_sentiment_textblob(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment
    return sentiment

In [20]:
# Apply the sentiment analysis function
df['sentiment'] = df['clean_email'].apply(predict_sentiment_textblob)

# Extract the sentiment label and score into separate columns
df['sentiment_blob_level'] = df['sentiment'].apply(lambda x: 'POSITIVE' if x.polarity > 0 else ('NEGATIVE' if x.polarity < 0 else 'NEUTRAL'))
df['sentiment_blob_score'] = df['sentiment'].apply(lambda x: x.polarity)

In [21]:
df.sentiment_blob_level.value_counts()

sentiment_blob_level
POSITIVE    2924
NEUTRAL     1278
NEGATIVE     699
Name: count, dtype: int64

In [23]:
d = dtale.show(df)
d.open_browser()

In [47]:
# from flair.models import TextClassifier
# from flair.data import Sentence

### NLTK

In [49]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

In [50]:
# Download the VADER lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\pis05408.PINNACLE\AppData\Roaming\nltk_data..
[nltk_data]     .


In [51]:
# Function to predict sentiment using VADER
def predict_sentiment_nltk(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment

In [52]:
# Apply the sentiment analysis function
df['sentiment_NLTK'] = df['clean_email'].apply(predict_sentiment_nltk)

# Extract the sentiment label and score into separate columns
df['sentiment_nltk_label'] = df['sentiment_NLTK'].apply(lambda x: 'POSITIVE' if x['compound'] >= 0.05 else ('NEGATIVE' if x['compound'] <= -0.05 else 'NEUTRAL'))
df['sentiment_nltk_score'] = df['sentiment_NLTK'].apply(lambda x: x['compound'])

In [54]:
df['sentiment_nltk_label'].value_counts()

sentiment_nltk_label
POSITIVE    4009
NEUTRAL      564
NEGATIVE     328
Name: count, dtype: int64

### SpaCy

In [63]:
# import spacy
# spacy.cli.download("en_core_web_sm")

In [60]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

In [64]:
# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

# Add the TextBlob component to the SpaCy pipeline
nlp.add_pipe("spacytextblob")

In [65]:
# Function to predict sentiment using SpaCy
def predict_sentiment_spacy(text):
    doc = nlp(text)
    sentiment = {
        'label': 'POSITIVE' if doc._.polarity > 0 else ('NEGATIVE' if doc._.polarity < 0 else 'NEUTRAL'),
        'score': doc._.polarity
    }
    return sentiment

In [66]:
# Apply the sentiment analysis function
df['sentiment_spacy'] = df['clean_email'].apply(predict_sentiment_spacy)

# Extract the sentiment label and score into separate columns
df['sentiment_spacy_label'] = df['sentiment_spacy'].apply(lambda x: x['label'])
df['sentiment_spacy_score'] = df['sentiment_spacy'].apply(lambda x: x['score'])

In [69]:
# df.columns

In [68]:
df['sentiment_spacy_label'].value_counts()

sentiment_spacy_label
POSITIVE    2924
NEUTRAL     1278
NEGATIVE     699
Name: count, dtype: int64

### Transformer 

In [71]:
# pip install transformers torch


In [72]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

In [73]:
# Initialize the sentiment analysis pipeline with a pre-trained RoBERTa model
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

c:\Suraj\projects\test\env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.



In [74]:
# Define the sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Function to predict sentiment
def predict_sentiment(text):
    result = sentiment_pipeline(text)
    return result[0] if result else {'label': 'UNKNOWN', 'score': 0.0}

In [76]:
# Apply the sentiment analysis function
df['sentiment_cardiffnlp'] = df['clean_email'].apply(predict_sentiment)

# Extract the sentiment label and score into separate columns
df['sentiment_label_cardiffnlp'] = df['sentiment_cardiffnlp'].apply(lambda x: x['label'])
df['sentiment_score_cardiffnlp'] = df['sentiment_cardiffnlp'].apply(lambda x: x['score'])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Suraj\projects\test\env\lib\site-packages\IPython\core\interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\pis05408.PINNACLE\AppData\Local\Temp\ipykernel_24332\538123373.py", line 2, in <module>
    df['sentiment_cardiffnlp'] = df['clean_email'].apply(predict_sentiment)
  File "c:\Suraj\projects\test\env\lib\site-packages\pandas\core\series.py", line 4753, in apply
    axis=_shared_doc_kwargs["axis"],
  File "c:\Suraj\projects\test\env\lib\site-packages\pandas\core\apply.py", line 1207, in apply
    return res
  File "c:\Suraj\projects\test\env\lib\site-packages\pandas\core\apply.py", line 1287, in apply_standard
    ) -> Callable[[npt.NDArray, Index, Index], dict[int, Any]]:
  File "c:\Suraj\projects\test\env\lib\site-packages\pandas\core\base.py", line 921, in _map_values
    return algorithms.map_array(arr, mapper, na_action=na_action, convert=convert)
  File "c:\Suraj\proje

### AllenNLP

In [78]:
# pip install allennlp allennlp-models


In [80]:
from allennlp.predictors.predictor import Predictor

In [81]:
# Load the sentiment analysis predictor
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/sst-roberta-large-2020.02.17.tar.gz")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pis05408.PINNACLE\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pis05408.PINNACLE\AppData\Roaming\nltk_data..
[nltk_data]     .
2024-05-23 19:54:01,915 - INFO     - Plugin allennlp_models available
2024-05-23 19:54:02,508 - INFO     - https://storage.googleapis.com/allennlp-public-models/sst-roberta-large-2020.02.17.tar.gz not found in cache, downloading to C:\Users\pis05408.PINNACLE\.allennlp\cache\d4cca2cbc54d5b1ed6450dad2c2ee3df941d1f64b94ed7c29ee92b530ed32975.68c2d52dd7bf3cba019951262c2624faa0a0e4977a16c62b59167cb9247985f0


c:\Suraj\projects\test\env\lib\site-packages\rich\live.py:229: UserWarning:

install "ipywidgets" for Jupyter support

2024-05-23 19:56:06,746 - INFO     - loading archive file https://storage.googleapis.com/allennlp-public-models/sst-roberta-large-2020.02.17.tar.gz from cache at C:\Users\pis05408.PINNACLE\.allennlp\cache\d4cca2cbc54d5b1ed6450dad2c2ee3df941d1f64b94ed7c29ee92b530ed32975.68c2d52dd7bf3cba019951262c2624faa0a0e4977a16c62b59167cb9247985f0
2024-05-23 19:56:06,747 - INFO     - extracting archive file C:\Users\pis05408.PINNACLE\.allennlp\cache\d4cca2cbc54d5b1ed6450dad2c2ee3df941d1f64b94ed7c29ee92b530ed32975.68c2d52dd7bf3cba019951262c2624faa0a0e4977a16c62b59167cb9247985f0 to temp dir C:\Users\PIS054~1.PIN\AppData\Local\Temp\tmp82t24yl3
2024-05-23 19:56:14,765 - WARNING  - error loading _jsonnet (this is expected on Windows), treating C:\Users\PIS054~1.PIN\AppData\Local\Temp\tmp82t24yl3\config.json as plain json
2024-05-23 19:56:14,774 - INFO     - dataset_reader.type = sst_tokens
2024-05-23 19:56:14,775 - INFO     - dataset_reader.max_instances = None
2024-05-23 19:56:14,775 - INFO     - dataset

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Suraj\projects\test\env\lib\site-packages\transformers\utils\import_utils.py", line 1535, in _get_module
  File "c:\Suraj\projects\test\env\lib\importlib\__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 984, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'transformers.models.align.configuration_align'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Suraj\projects\test\env\lib\site-packages\IPython\core\interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\pis05408.PINNACLE\AppData\Local\Temp\ipykernel_24332\722857420.py", line 2, in <module>
    predictor = Predic

In [ ]:
# Function to predict sentiment using AllenNLP
def predict_sentiment_allennlp(text):
    result = predictor.predict(sentence=text)
    label = 'POSITIVE' if result['label'] == '1' else 'NEGATIVE'
    score = result['probs'][int(result['label'])]
    return {'label': label, 'score': score}

In [ ]:
# Apply the sentiment analysis function
df['sentiment_allennlp'] = df['clean_email'].apply(predict_sentiment_allennlp)

# Extract the sentiment label and score into separate columns
df['sentiment_label_allennlp'] = df['sentiment_allennlp'].apply(lambda x: x['label'])
df['sentiment_score_allennlp'] = df['sentiment_allennlp'].apply(lambda x: x['score'])